In [1]:
import tensorflow as tf
tf.__version__

'2.7.0'

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices() # 실행 가능한 cpu, gpu 목록

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3735716130214030674
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1398373991
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2874555411126317602
 physical_device_desc: "device: 0, name: NVIDIA GeForce GT 1030, pci bus id: 0000:01:00.0, compute capability: 6.1"
 xla_global_id: 416903419]

In [4]:
gpus=tf.config.experimental.list_physical_devices('GPU') # 실행 가능한 gpu 목록
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
# 기본적으로 gpu로 실행됨
tf.debugging.set_log_device_placement(True)

a=tf.constant([[1,2,3],[4,5,6]])
b=tf.constant([[1,2],[3,4],[5,6]])
c=tf.matmul(a,b)
print(c)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[22 28]
 [49 64]], shape=(2, 2), dtype=int32)


In [3]:
tf.debugging.set_log_device_placement(True)
# cpu에서 실행시키는 방법
with tf.device('/CPU:0'):
    a=tf.constant([[1,2,3],[4,5,6]])
    b=tf.constant([[1,2],[3,4],[5,6]])
    c=tf.matmul(a,b)
    print(c)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[22 28]
 [49 64]], shape=(2, 2), dtype=int32)


In [2]:
tf.debugging.set_log_device_placement(True)
# gpu에서 실행시키는 방법
with tf.device('/GPU:0'):
    a=tf.constant([[1,2,3],[4,5,6]])
    b=tf.constant([[1,2],[3,4],[5,6]])
    c=tf.matmul(a,b)
    print(c)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[22 28]
 [49 64]], shape=(2, 2), dtype=int32)


In [4]:
# gpu memory는 기본적으로 full size로 설정되므로 gpu 메모리를 탄력적으로 사용하는 방법

# 텐서플로 v1에서 gpu의 메모리를 설정하는 방법
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

config=tf.ConfigProto()
config.gpu_options.allow_growth=True # gpu 메모리를 점진적으로 증가시키는 방법
config.gpu_options.per_process_gpu_memory_fraction=0.4 # gpu 메모리의 40%만 할당하는 방법
session=tf.Session(config=config)


In [1]:
# 텐서플로 v2에서 gpu의 메모리를 설정하는 방법
import tensorflow as tf

gpus=tf.config.experimental.list_physical_devices('GPU')
print(gpus)
# 메모리 사용량을 직접 지정하는 방법
tf.config.experimental.set_virtual_device_configuration(gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=512)])

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import tensorflow as tf
import sys
print(sys.version)
print(tf.__version__)

3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]
2.7.0


In [3]:
# 상수 선언
hello=tf.constant("Hello World")
print(hello)
print(hello.numpy())
print(hello.numpy().decode('utf-8')) # str 클래스로 변환, b가 사라짐 

a=tf.constant(10)
b=tf.constant(32)
c=tf.add(a, b) # c = a + b
print(c.numpy())
print([a.numpy(),b.numpy(),c.numpy()])

tf.Tensor(b'Hello World', shape=(), dtype=string)
b'Hello World'
Hello World
42
[10, 32, 42]


In [3]:
node1=tf.constant(3.0)
node2=tf.constant(4.0)
node3=tf.add(node1,node2)
print(node3.numpy())

7.0


In [4]:
def output(a,b):
    return tf.multiply(a,b)

print(output(3.0,5.0).numpy())
print(output(0.0,6.0).numpy())
print(output([2.0,3.1,7.4],[6.0,2.5,3.1]).numpy())

15.0
0.0
[12.    7.75 22.94]


In [5]:
# 텐서플로의 변수 선언
W=tf.Variable([0.3], dtype=tf.float32)
b=tf.Variable([-0.3], dtype=tf.float32)

def linear_model(x):
    return tf.multiply(W, tf.add(x,b))

x=[1.,2.,3.,4.]
linear_model(x).numpy() # x의 값에 따라 linear_model 계산

array([0.21000001, 0.51000005, 0.81000006, 1.11      ], dtype=float32)

In [6]:
import numpy as np

X=np.array([[1,2,3],[4,5,6]])
print('X\n',X)
# 3행 2열의 랜덤
W=np.array([[np.random.randint(10) for col in range(2)] for row in range(3)])
print('W\n',W)
b=np.array([[np.random.randint(10) for col in range(1)] for row in range(2)])
print('b\n',b)

def expr(X):
    return tf.matmul(X,W) + b # 행렬 곱셈

print('Result\n', expr(X).numpy())

X
 [[1 2 3]
 [4 5 6]]
W
 [[7 3]
 [1 2]
 [4 7]]
b
 [[5]
 [7]]
Result
 [[26 33]
 [64 71]]


In [7]:
import tensorflow.compat.v1 as tf # v1 활성화
tf.disable_v2_behavior() # v2 비활성화

def gugu(dan):
    # 플레이스홀더 선언 (그래프에 입력할 변수)
    left=tf.placeholder(tf.int32)
    right=tf.placeholder(tf.int32)
    calc=tf.multiply(left, right) # 그래프의 수식

    with tf.Session() as sess: # 세션이 생성됨
        for i in range(1,10):
            result=sess.run(calc, feed_dict={left: dan, right: i})
            # 세션.run(실행할 함수, feed_dict={변수: 입력할 값})
            print(f'{dan} × {i} = {result}')

gugu(7)

Instructions for updating:
non-resource variables are not supported in the long term
7 × 1 = 7
7 × 2 = 14
7 × 3 = 21
7 × 4 = 28
7 × 5 = 35
7 × 6 = 42
7 × 7 = 49
7 × 8 = 56
7 × 9 = 63


In [1]:
import tensorflow as tf

def calc(left, right):
    return tf.multiply(left, right)

def gugu(dan):
    for i in range(1,10):
        result=calc(dan,i)
        print(f'{dan} × {i} = {result}')

gugu(7)

7 × 1 = 7
7 × 2 = 14
7 × 3 = 21
7 × 4 = 28
7 × 5 = 35
7 × 6 = 42
7 × 7 = 49
7 × 8 = 56
7 × 9 = 63
